In [1]:
import pandas as pd

In [2]:
import os
import pandas as pd
from tkinter import Tk, filedialog, StringVar, Toplevel, Label, Button, OptionMenu, messagebox

# Variables globales para almacenar los DataFrames
global_main_data = None
global_maestro_data = None

def process_and_concat_excel_files():
    global global_main_data, global_maestro_data
    
    # Suppress the root Tkinter window
    root = Tk()
    root.withdraw()

    # Select the folder containing Excel files
    folder_path = filedialog.askdirectory(
        title="Select Folder with Excel Files"
    )

    if not folder_path:
        print("No folder selected.")
        return None, None

    # List to store DataFrames
    dataframes = []
    maestro_dataframes = []

    # Iterate through files in the selected folder
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.xlsx', '.xls')):
            file_path = os.path.join(folder_path, filename)
            
            try:
                # Load sheet names from the current file
                excel_file = pd.ExcelFile(file_path)
                sheet_names = excel_file.sheet_names
                
                # Dynamically select a sheet
                def select_sheet():
                    # Create a new top-level window
                    selector_window = Toplevel(root)
                    selector_window.title(f"Select Sheet for {filename}")

                    # Label
                    Label(selector_window, text=f"Select a sheet for {filename}:").pack(pady=10)

                    # Dropdown menu for sheet names
                    selected_sheet = StringVar(selector_window)
                    selected_sheet.set(sheet_names[0])  # Set default value
                    sheet_menu = OptionMenu(selector_window, selected_sheet, *sheet_names)
                    sheet_menu.pack(pady=10)

                    # Variable to store the selected sheet
                    selected_df = [None]

                    # Button to confirm selection
                    def load_data():
                        sheet_name = selected_sheet.get()
                        try:
                            # Load data from the selected sheet
                            df = pd.read_excel(file_path, sheet_name=sheet_name, dtype={'Centro': 'str'})
                            
                            # Add a column to track the source file
                            df['Source_File'] = filename
                            
                            selected_df[0] = df
                            print(f"\nData loaded successfully from {filename}, sheet: {sheet_name}")
                            selector_window.destroy()
                        except Exception as e:
                            print(f"Error loading sheet in {filename}: {e}")
                            messagebox.showerror("Error", f"Could not load sheet: {e}")

                    Button(selector_window, text="Load Sheet", command=load_data).pack(pady=10)

                    # Wait for the window to be closed
                    selector_window.wait_window()

                    return selected_df[0]

                # Load the main dataframe
                main_df = select_sheet()
                if main_df is not None:
                    dataframes.append(main_df)

                # Try to load Maestro sheet
                maestro_sheets = [sheet for sheet in sheet_names if 'maestro' in sheet.lower()]
                if maestro_sheets:
                    try:
                        maestro_df = pd.read_excel(file_path, sheet_name=maestro_sheets[0], header=1)
                        maestro_df['Source_File'] = filename
                        maestro_dataframes.append(maestro_df)
                        print(f"Maestro sheet loaded for {filename}")
                    except Exception as e:
                        print(f"Could not load Maestro sheet in {filename}: {e}")

            except Exception as e:
                print(f"Error processing {filename}: {e}")

    # Concatenate DataFrames
    if dataframes:
        try:
            # Concatenate all loaded DataFrames
            global_main_data = pd.concat(dataframes, ignore_index=True)
            
            print("\nDataFrame Concatenation Complete:")
            print(global_main_data.info())

            # Concatenate Maestro DataFrames if available
            if maestro_dataframes:
                global_maestro_data = pd.concat(maestro_dataframes, ignore_index=True)
                print("\nMaestro DataFrame Concatenation Complete:")
                print(global_maestro_data.info())

            return global_main_data, global_maestro_data

        except Exception as e:
            print(f"Error concatenating DataFrames: {e}")
            messagebox.showerror("Concatenation Error", str(e))
    else:
        print("No DataFrames to concatenate.")
        messagebox.showwarning("Warning", "No files were processed.")

    # Close the root window
    root.destroy()

# Llamada a la función para procesar archivos
process_and_concat_excel_files()

# Ahora puedes acceder a los DataFrames globales en cualquier parte del script
def use_dataframes():
    # Verificar si los DataFrames están disponibles
    if 'global_main_data' in globals() and global_main_data is not None:
        print("\nMain DataFrame:")
        print(global_main_data.head())
        print(global_main_data.columns)
        
        # Ejemplo de filtrado
        # filtrado = global_main_data[global_main_data['Source_File'] == 'archivo_especifico.xlsx']
    
    if 'global_maestro_data' in globals() and global_maestro_data is not None:
        print("\nMaestro DataFrame:")
        print(global_maestro_data.head())
        print(global_maestro_data.columns)

# Puedes llamar a esta función después de procesar los archivos
# use_dataframes()



No folder selected.


In [52]:
global_main_data

,Centro,Almacén,Clase doc.ventas,Nº pedido cliente,Documento comercial,Fecha documento,Nombre 1,destinatario de mercancía,Solicitante,Cod.Dest.Mercancia,...,Fecha de precio,Creado el,Hora,VIN / Patente,Creado por,Codigo Marca,Descripcion Marca.,Fecha salida mcías.,Fecha de entrega,Source_File
0,1305,1010.0,Z300,AGM 15004266,1652483,2024-11-23,FERNANDO GOMEZ Y COMPAÑIA LIMITADA,FERNANDO GOMEZ PAICAVI,79873420-2,2023,...,2024-11-23,2024-11-23,15:00:41,NaN,RBELTRAN,20.0,DFSK,2024-11-25,2024-11-25,Pedidos DFSK Semana 47.XLSX
1,1305,1010.0,Z300,PZA OESTE,1653877,2024-11-22,POMPEYO CARRASCO SPA,POMPEYO PLAZA OESTE,81318700-0,2079,...,2024-11-22,2024-11-22,17:22:27,NaN,CLACVARGAS,20.0,DFSK,2024-11-22,2024-11-22,Pedidos DFSK Semana 47.XLSX
2,1305,1010.0,Z300,PZA OESTE,1653877,2024-11-22,POMPEYO CARRASCO SPA,POMPEYO PLAZA OESTE,81318700-0,2079,...,2024-11-22,2024-11-22,17:22:27,NaN,CLACVARGAS,20.0,DFSK,2024-11-22,2024-11-22,Pedidos DFSK Semana 47.XLSX
3,1305,1010.0,Z300,PZA OESTE,1653877,2024-11-22,POMPEYO CARRASCO SPA,POMPEYO PLAZA OESTE,81318700-0,2079,...,2024-11-22,2024-11-22,17:22:27,NaN,CLACVARGAS,20.0,DFSK,2024-11-22,2024-11-22,Pedidos DFSK Semana 47.XLSX
4,1305,1010.0,Z300,NaN,1653714,2024-11-22,INCHCAPE COMERCIAL CHILE SA,INCHCAPE BUSTAMANTE,76620944-0,2642,...,2024-11-22,2024-11-22,13:40:53,NaN,PNAVARRON,20.0,DFSK,2024-11-22,2024-11-22,Pedidos DFSK Semana 47.XLSX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2177,1335,1010.0,Z300,SNOTRPA,1643212,2024-11-18,KIA CHILE SPA,INDUMOTORA ONE ENEA,79567420-9,2665,...,2024-11-18,2024-11-18,15:41:25,NaN,CLBJDIAZ,11.0,SUBARU,2024-11-18,2024-11-18,Pedidos SBR Semana 47.XLSX
2178,1335,1020.0,Z300,SNOTRPA,1643212,2024-11-18,KIA CHILE SPA,INDUMOTORA ONE ENEA,79567420-9,2665,...,2024-11-18,2024-11-18,15:41:25,NaN,CLBJDIAZ,11.0,SUBARU,2024-11-18,2024-11-18,Pedidos SBR Semana 47.XLSX
2179,1335,1010.0,Z300,SNOTRPA,1643212,2024-11-18,KIA CHILE SPA,INDUMOTORA ONE ENEA,79567420-9,2665,...,2024-11-18,2024-11-18,15:41:25,NaN,CLBJDIAZ,11.0,SUBARU,2024-11-18,2024-11-18,Pedidos SBR Semana 47.XLSX
2180,1335,1010.0,Z300,OTRODPA,1588315,2024-11-18,AUTOMOTRIZ CORDILLERA SA AUTOMOTRIZ,CECOR JORGE ALESSANDRI 18,79853470-K,2015,...,2024-11-18,2024-11-18,14:51:32,NaN,CLBJDIAZ,11.0,SUBARU,2024-11-18,2024-11-18,Pedidos SBR Semana 47.XLSX


In [53]:
inchcape = global_main_data

columns = [
    "Centro", "Clase doc.ventas", "Fecha documento","Nº pedido cliente", "Material", "Número de pieza del fabricante", "Denominación", "Cantidad de pedido"
]
inchcape = inchcape[columns]

In [54]:
inchcape.rename(columns={

    "Centro":"Ce.", "Clase doc.ventas":"ClVt", "Fecha documento":"Fecha doc-","Nº pedido cliente":"Pedido",  "Número de pieza del fabricante": "NPF", "Denominación":"Denominación", "Cantidad de pedido":"Ctd.ped..1"}, inplace=True)

C:\Users\lravlic\AppData\Local\Temp\ipykernel_19828\1595450971.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inchcape.rename(columns={


In [55]:
maestro= pd.read_excel('Copy of Pedido FR Semana 44..xlsx', sheet_name = 'Maestro 2.0', header=1)

In [56]:
maestro = maestro[['Último Eslabón y Material SAP','Familia','Segmentación ','Modelo']]

In [57]:
maestro.drop_duplicates(subset='Último Eslabón y Material SAP', inplace=True)

In [58]:
inchcape = inchcape.merge(maestro, left_on='Material', right_on='Último Eslabón y Material SAP', how='left')

In [59]:
inchcape.drop(columns={'Último Eslabón y Material SAP'}, inplace=True)

In [60]:
inchcape.head()

,Ce.,ClVt,Fecha doc-,Pedido,Material,NPF,Denominación,Ctd.ped..1,Familia,Segmentación,Modelo
0,1305,Z300,2024-11-23,AGM 15004266,2000009036,3774010-SA05,TELECOMANDO LUCES SUV 560,1,NaN,NaN,NaN
1,1305,Z300,2024-11-22,PZA OESTE,2000002310,1109120-02,FILTRO AIRE,5,NaN,NaN,NaN
2,1305,Z300,2024-11-22,PZA OESTE,2000005242,1109120-SA01,FILTRO AIRE SFG15T-16,3,NaN,NaN,NaN
3,1305,Z300,2024-11-22,PZA OESTE,2000005913,3611800K0000,SENSOR FASE GLORY SUV,2,NaN,NaN,NaN
4,1305,Z300,2024-11-22,NaN,2000002024,1600200A0300,DISCO DE EMBRAGUE 1.1,1,NaN,NaN,NaN


In [61]:
#Marca
mappings = {
    '1335': "Subaru",
    '1305': "DFSK",
    '1344': "Geely"
}

# Apply the mapping to create the 'Marca' column
inchcape['Marca'] = inchcape['Ce.'].map(mappings).fillna(0)


In [83]:
vor_dfsk = pd.read_excel("C:/Users/lravlic/Downloads/BD_VOR_EC.xlsx", sheet_name = 'DFSK', header=7)
vor_geely = pd.read_excel("C:/Users/lravlic/Downloads/BD_VOR_EC.xlsx",sheet_name = 'GEELY')
vor_subaru = pd.read_excel("C:/Users/lravlic/Downloads/BD_VOR_EC.xlsx",sheet_name = 'SUBARU', header = 7)

c:\Users\lravlic\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:223: UserWarning: Cell AG1755 is marked as a date but the serial value 1231347014 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
c:\Users\lravlic\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:223: UserWarning: Cell AG4931 is marked as a date but the serial value 1541873130 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
c:\Users\lravlic\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:223: UserWarning: Cell AG4938 is marked as a date but the serial value 1541873130 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
c:\Users\lravlic\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:223: UserWarning: Cell AG4939 is marked as a date but the serial value 1541873130 is 

In [82]:
vor_subaru.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60
0,Estado_VOR,N_VOR,VIN,MODELO,AÑO,NPF,MATERIAL,QTY,Descripción,Dealer,...,Columna4,Columna5,Columna6,Columna7,Columna8,Columna9,Columna10,Columna11,Columna12,Columna13
1,Cerrado,VOR1,73531FJ010,NaN,NaN,73531FJ010,100017303,1,VALVULAEXPANSION,LARRAIN Y VALDES LTDA. - BILBAO 5997,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cerrado,VOR2,4S4WX9JD6A4400629,NaN,NaN,12108AA970,100005427,1,JGOMETALBIELA,INDUMOTORA ONE S.A. - CdR - 10 DE JULIO 402,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cerrado,VOR2,4S4WX9JD6A4400629,NaN,NaN,12209AA770,100005527,1,JGOMETALBCDA,INDUMOTORA ONE S.A. - CdR - 10 DE JULIO 402,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cerrado,VOR3,4S4WX9JD8C4402336,NaN,NaN,62124XA002,100001191,1,TIRANTEPUERTADEL,AUTOMOTRIZ CORDILLERA S.A. - CAUPOLICAN 1476,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Cerrado,VOR4,JF1BN9LC2HG017698,NaN,NaN,84001AL070,100021059,1,OPTICOIZQUIERDO,AUTOMOTRIZ CORDILLERA S.A. - CAUPOLICAN 1476,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Cerrado,VOR5,JF1GVFLV5BG014317,NaN,NaN,56410AG005,100007779,1,CUBREMOTOR,MACH S.A. - AV. LIBERTAD 1080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Cerrado,VOR6,JF1BN9LC2FG002359,NaN,NaN,84912AL110,100020835,1,FAROLCOMBINADOTRAS.RH,COM. E INV. AUTOMOTRIZ ALAMEDA LTDA - ALAMEDA ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Cerrado,VOR7,JF1BP9LLA6G053133,NaN,NaN,93033AG032,100011936,1,LOGODELANTERO,MACH S.A. - AV. LIBERTAD 1080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Cerrado,VOR8,JF1SJ5LA3FG216089,NaN,NaN,57229SG0009P,100016927,1,CAPO,POMPEYO CARRASCO SPA - CAMINO MELIPILLA 11585,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
inchcape['ClVt'].value_counts()

ClVt
Z300    1743
Z303     225
Z220     121
Z310      30
Z238      22
Z400      21
Z319      14
Z235       4
Z304       2
Name: count, dtype: int64

In [63]:
inchcape['Líneas OK'] = inchcape['ClVt'].apply(lambda x: 1 if x == "Z300" else 0)
inchcape['Líneas VOR'] = inchcape['ClVt'].apply(lambda x: 1 if x == "ZVOR" else 0)


In [64]:
inchcape['Líneas Totales'] = inchcape['Líneas VOR']+inchcape['Líneas OK'] 

In [65]:
inchcape['Qty OK'] = inchcape['Líneas OK'] * inchcape['Ctd.ped..1']

In [66]:
inchcape['Qty VOR'] = inchcape['Líneas VOR'] * inchcape['Ctd.ped..1']

marca
vor
li ok
li vor
li totales
qty
qty
qty
segmentacion (maestro)
modelo (maestro)
familia (maestro)


In [67]:
inchcape['Qty VOR'] = inchcape['Líneas VOR'] * inchcape['Ctd.ped..1']

In [68]:
inchcape.groupby('Marca').agg({'Líneas OK':'sum',
                               'Líneas VOR':'sum',
                               'Líneas Totales':'sum'})

,Líneas OK,Líneas VOR,Líneas Totales
Marca,,,
0,0,0,0
DFSK,239,0,239
Geely,47,0,47
Subaru,1457,0,1457


In [17]:
nnss =pd.read_csv('consolidado sem 48.csv')

C:\Users\lravlic\AppData\Local\Temp\ipykernel_16260\450141765.py:1: DtypeWarning: Columns (7,16,21,23,26,29,32,33,44) have mixed types. Specify dtype option on import or set low_memory=False.
  nnss =pd.read_csv('consolidado sem 48.csv')


In [18]:
nnss['Semana'].value_counts()

Semana
34    11481
2     11442
35    10220
3     10165
23    10151
4     10069
14     9988
45     9724
6      9665
43     9622
28     9508
10     9503
37     9302
5      9283
9      9267
20     9236
41     9170
16     9112
27     9087
8      9075
12     9051
11     9013
24     9003
30     8976
7      8945
39     8766
40     8742
31     8718
15     8702
19     8680
32     8646
47     8496
46     8430
17     8380
22     8346
26     8047
1      7506
18     7299
29     7271
33     7264
21     6986
25     6351
13     5978
42     5099
36     4978
44     4722
38     2730
Name: count, dtype: int64

In [19]:
meses = {
    "ene": 1,
    "feb": 2,
    "mar": 3,
    "abr": 4,
    "may": 5,
    "jun": 6,
    "jul": 7,
    "ago": 8,
    "sep": 9,
    "oct": 10,
    "nov": 11,
    "dic": 12
}

# Cambiar valores de la columna 'Mes' usando el diccionario
inchcape['Mes'] = inchcape['Fecha doc-'].dt.month
inchcape['Semana'] =   inchcape['Fecha doc-'].dt.isocalendar().week


In [20]:
nnss['Centro'] = ''
nnss['NPF'] = ""
nnss['VOR'] = ""
nnss['Considerar'] = ""

inchcape.rename(columns= {'Ce.':'Centro','ClVt':'Clase doc.ventas','Pedido':'Documento de ventas','Denominación':'Descripción del material','Ctd.ped..1':'Cantidad de pedido (Posición)','Marca':'Nombre sector','Líneas OK':'Línea OK', 'Líneas VOR':'Línea VOR', 'Líneas Totales':'Línea Totales','Qty OK':'Qty Venta' ,'Qty Totales':'Qty Total','Fecha doc-':'Fecha documento'}, inplace=True)




In [21]:
inchcape.columns

Index(['Centro', 'Clase doc.ventas', 'Fecha documento', 'Documento de ventas',
       'Material', 'NPF', 'Descripción del material',
       'Cantidad de pedido (Posición)', 'Familia', 'Segmentación ', 'Modelo',
       'Nombre sector', 'Línea OK', 'Línea VOR', 'Línea Totales', 'Qty Venta',
       'Qty VOR', 'Mes', 'Semana'],
      dtype='object')

In [22]:
consolidado = pd.concat([nnss,inchcape])

In [23]:
nnss['Mes']

0         11
1         11
2         11
3         11
4         11
          ..
400190    10
400191    10
400192    10
400193    10
400194    10
Name: Mes, Length: 400195, dtype: int64

In [29]:
consolidado.columns

Index(['Unnamed: 0', 'Referencia de cliente', 'Fecha documento',
       'Clase doc.ventas', 'Documento de ventas', 'Posición', 'AUX',
       'Solicitante', 'Nombre Solicitante', 'Cantidad de pedido (Posición)',
       'Un.medida venta', 'Valor neto (posición)', 'Material',
       'Descripción del material', 'Material R3', 'UE R3',
       'Nombre Sector Anterior2', 'Nombre sector', 'Origen', 'Modelo',
       'Familia', 'Parque', 'Apretura Parque', 'Plan Mantención',
       'Clase R (actual)', 'Segmentación', 'Segmentación N-5', 'Estratégicos',
       'Planificables', 'Planificable N-5', 'Equipo', 'Flag IPL', 'Incidencia',
       'Causa Raíz', 'Línea OK', 'Línea VOR', 'Línea Totales', 'Qty Venta',
       'Qty VOR', 'Qty Total', 'Semana', 'Día', 'Mes', 'Año',
       'Moneda del documento', 'Centro', 'NPF', 'VOR', 'Considerar',
       'Segmentación '],
      dtype='object')

143

In [22]:
nnss['Centro'] = ''


In [34]:
consolidado.head()

,Unnamed: 0,Referencia de cliente,Fecha documento,Clase doc.ventas,Documento de ventas,Posición,AUX,Solicitante,Nombre Solicitante,Cantidad de pedido (Posición),...,Semana,Día,Mes,Año,Moneda del documento,Centro,NPF,VOR,Considerar,Segmentación
0,0.0,Valpo ES,2024-11-18,ZRDP,6366303,10.0,636630310.0,286808,DISTRIBUIDORA DE VEHICULOS SUZUVAL,1,...,47,18.0,11,2024.0,NaN,,,,,NaN
1,1.0,Valpo ES,2024-11-18,ZRDP,6366303,20.0,636630320.0,286808,DISTRIBUIDORA DE VEHICULOS SUZUVAL,1,...,47,18.0,11,2024.0,NaN,,,,,NaN
2,2.0,Valpo ES,2024-11-18,ZRDP,6366304,10.0,636630410.0,286808,DISTRIBUIDORA DE VEHICULOS SUZUVAL,1,...,47,18.0,11,2024.0,NaN,,,,,NaN
3,3.0,Valpo ES,2024-11-18,ZRDP,6366305,10.0,636630510.0,286808,DISTRIBUIDORA DE VEHICULOS SUZUVAL,1,...,47,18.0,11,2024.0,NaN,,,,,NaN
4,4.0,Valpo ES,2024-11-18,ZRDP,6366306,10.0,636630610.0,286808,DISTRIBUIDORA DE VEHICULOS SUZUVAL,1,...,47,18.0,11,2024.0,NaN,,,,,NaN


In [35]:
consolidado.to_csv('consolidado_legacies.csv')

In [221]:
final = pd.read_csv('consolidado_legacies.csv')

C:\Users\lravlic\AppData\Local\Temp\ipykernel_10228\3849998864.py:1: DtypeWarning: Columns (2,5,8,9,11,15,16,17,19,22,23,24,26,27,29,30,31,33,34,45,47,50) have mixed types. Specify dtype option on import or set low_memory=False.
  final = pd.read_csv('consolidado_legacies.csv')


In [25]:
consolidado

,Unnamed: 0.1,Unnamed: 0,Referencia de cliente,Fecha documento,Clase doc.ventas,Documento de ventas,Posición,AUX,Solicitante,Nombre Solicitante,...,Semana,Día,Mes,Año,Moneda del documento,Centro,NPF,VOR,Considerar,Segmentación
0,0.0,0.0,Valpo ES,2024-11-11,ZRDP,6358237,10.0,635823710.0,286808,DISTRIBUIDORA DE VEHICULOS SUZUVAL,...,46,11.0,11,2024.0,NaN,,,,,NaN
1,1.0,1.0,Valpo ES,2024-11-11,ZRDP,6358237,20.0,635823720.0,286808,DISTRIBUIDORA DE VEHICULOS SUZUVAL,...,46,11.0,11,2024.0,NaN,,,,,NaN
2,2.0,2.0,MCR VARIOS,2024-11-11,ZRDP,6358238,10.0,635823810.0,165,SERVICIOS AUTOMOTRICES ANSA SPA,...,46,11.0,11,2024.0,NaN,,,,,NaN
3,3.0,3.0,MCR VARIOS,2024-11-11,ZRDP,6358238,20.0,635823820.0,165,SERVICIOS AUTOMOTRICES ANSA SPA,...,46,11.0,11,2024.0,NaN,,,,,NaN
4,4.0,4.0,MCR VARIOS,2024-11-11,ZRDP,6358238,30.0,635823830.0,165,SERVICIOS AUTOMOTRICES ANSA SPA,...,46,11.0,11,2024.0,NaN,,,,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2330,NaN,NaN,NaN,2024-11-17 00:58:24,ZVOR,VOR09603,NaN,NaN,NaN,NaN,...,46,NaN,11,NaN,NaN,1344,5035041700661,NaN,NaN,CC
2331,NaN,NaN,NaN,2024-11-17 00:58:24,ZVOR,VOR09603,NaN,NaN,NaN,NaN,...,46,NaN,11,NaN,NaN,1344,5035041600661,NaN,NaN,CD
2332,NaN,NaN,NaN,2024-11-17 00:58:24,ZVOR,VOR09603,NaN,NaN,NaN,NaN,...,46,NaN,11,NaN,NaN,1344,5083029300A10,NaN,NaN,BB
2333,NaN,NaN,NaN,2024-11-17 01:00:19,ZVOR,VOR09604,NaN,NaN,NaN,NaN,...,46,NaN,11,NaN,NaN,1344,3055014400,NaN,NaN,OO
